In [1]:
#requirments if not installed
# !pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu118
# !pip install opencv-python
# !pip install pycocotools
# !pip install cython

In [171]:
#imports
import torch
import cv2
from yolact import Yolact
from collections import defaultdict
import torch.backends.cudnn as cudnn
from utils.augmentations import FastBaseTransform
from utils import timer
from data import cfg
from layers.output_utils import postprocess
from data import COLORS

import tkinter as tk
from tkinter import ttk
import PIL.Image, PIL.ImageTk
from tkinter import BooleanVar, Scale
from PIL import Image, ImageTk

In [172]:
##SOME PARAMETERS ARGS
global net
global cuda_enabled
global sc
global top_k
global display_text_bbx_enabled
global img_scale
global class_list

class_list = set()
cuda_enabled = torch.cuda.is_available()
display_masks_enabled = cuda_enabled
img_scale=1
sc=0.45
top_k =15
color_cache = defaultdict(lambda: {})
display_text_bbx_enabled=True;

In [173]:
print('Loading model...', end='')
net = Yolact()
net.load_weights("model/yolact_base_54_800000.pth")
net.eval()
print(' Done.')

Loading model... Done.


In [174]:
#IF CUUDA added 
#FOR NOW IT WORKS ONLU ON CUDA MODE
def set_default():
    if cuda_enabled and torch.cuda.is_available():
        cudnn.fastest = True
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        cudnn.fastest = False
        torch.set_default_tensor_type('torch.FloatTensor')

In [175]:
#LOADING MODEL
def load_model():
    if cuda_enabled:
        net.cuda()
    else:
        net.cpu()

In [176]:
# Define a function to prepare and display segmentation results

def prep_display(dets_out, img, h, w, undo_transform=True, class_color=False, mask_alpha=0.45, fps_str=''):
    
    # Check if we need to undo image transformation
    if undo_transform:
        img_numpy = undo_image_transformation(img, w, h)  # Undo image transformation
        if cuda_enabled:
            img_gpu = torch.Tensor(img_numpy).cuda()  # Transfer to CPU
        else:
            img_gpu = torch.Tensor(img_numpy).cpu()  # Transfer to GPU
    else:
        img_gpu = img / 255.0  # Normalize image if not transformed
        h, w, _ = img.shape
    
    # Perform post-processing on detection results
    with timer.env('Postprocess'):
        save = cfg.rescore_bbox
        cfg.rescore_bbox = True
        t = postprocess(dets_out, w, h, visualize_lincomb=False,
                                        crop_masks=True,
                                        score_threshold=0)
        cfg.rescore_bbox = save
    
    with timer.env('Copy'):
        idx = t[1].argsort(0, descending=True)[:top_k]  # Sort and select top detections
        
        if cfg.eval_mask_branch:
            masks = t[3][idx]  # Extract masks if available
        classes, scores, boxes = [x[idx].cpu().numpy() for x in t[:3]]  # Extract classes, scores, and boxes

    # Determine the number of detections to consider
    num_dets_to_consider = min(top_k, classes.shape[0])
    for j in range(num_dets_to_consider):
        if scores[j] < sc:
            num_dets_to_consider = j
            break

    # Define a function to get color for class/category
    def get_color(j, on_gpu=None):
        global color_cache
        color_idx = (classes[j] * 5 if class_color else j * 5) % len(COLORS)
        
        if on_gpu is not None and color_idx in color_cache[on_gpu]:
            return color_cache[on_gpu][color_idx]
        else:
            color = COLORS[color_idx]
            if not undo_transform:
                # The image might come in as RGB or BRG, depending
                color = (color[2], color[1], color[0])
            if on_gpu is not None:
                color = torch.Tensor(color).to(on_gpu).float() / 255.
                color_cache[on_gpu][color_idx] = color
            return color

    if display_masks_enabled and cuda_enabled and cfg.eval_mask_branch and num_dets_to_consider > 0:
        # After this, mask is of size [num_dets, h, w, 1]
        masks = masks[:num_dets_to_consider, :, :, None]
        colors = torch.cat([get_color(j, on_gpu=img_gpu.device.index).view(1, 1, 1, 3) for j in range(num_dets_to_consider)], dim=0)
        masks_color = masks.repeat(1, 1, 1, 3) * colors * mask_alpha

        inv_alph_masks = masks * (-mask_alpha) + 1

        masks_color_summand = masks_color[0]
        if num_dets_to_consider > 1:
            inv_alph_cumul = inv_alph_masks[:(num_dets_to_consider-1)].cumprod(dim=0)
            masks_color_cumul = masks_color[1:] * inv_alph_cumul
            masks_color_summand += masks_color_cumul.sum(dim=0)

        img_gpu = img_gpu * inv_alph_masks.prod(dim=0) + masks_color_summand

    img_numpy = (img_gpu * 255).byte().cpu().numpy()

    if num_dets_to_consider == 0:
        return img_numpy

    if display_text_bbx_enabled:
        for j in reversed(range(num_dets_to_consider)):
            x1, y1, x2, y2 = boxes[j, :]
            color = get_color(j)
            score = scores[j]

            if True:
                cv2.rectangle(img_numpy, (x1, y1), (x2, y2), color, 1)
                
                _class = cfg.dataset.class_names[classes[j]]
                text_str = '%s: %.2f' % (_class, score) if True else _class
                class_list.add(_class)
                

                font_face = cv2.FONT_HERSHEY_DUPLEX
                font_scale = 0.6
                font_thickness = 1

                text_w, text_h = cv2.getTextSize(text_str, font_face, font_scale, font_thickness)[0]

                text_pt = (x1, y1 - 3)
                text_color = [255, 255, 255]

                cv2.rectangle(img_numpy, (x1, y1), (x1 + text_w, y1 - text_h - 4), color, -1)
                cv2.putText(img_numpy, text_str, text_pt, font_face, font_scale, text_color, font_thickness, cv2.LINE_AA)
    
    return img_numpy


In [177]:
# Define a function to evaluate an image using a segmentation model
def evaluateimg(img):
    
    # Convert the image to a PyTorch tensor and transfer it to the GPU
    if cuda_enabled:
        frame = torch.from_numpy(img).cuda().float()
    else:
        frame = torch.from_numpy(img).cpu().float()

    # Apply FastBaseTransform to prepare the image for model evaluation
    batch = FastBaseTransform()(frame.unsqueeze(0))
    
    # Disable gradient tracking for model inference
    with torch.no_grad():
        preds = net(batch)  # Pass the image through the segmentation model

    # Prepare the segmented image for display and return it
    res = prep_display(preds, frame, None, None, undo_transform=False)
    return res

    # Example of how to use the 'evaluateimg' function
    image = evaluateimg(frame)  # 'frame' is assumed to be an input image here


In [178]:
def setting():
    set_default()
    load_model()

In [179]:
# test code

'''

setting()
# Open the default camera (camera index 0)
cap = cv2.VideoCapture(0)
# Continuously capture frames from the webcam
while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Check if the frame was successfully captured
    if not ret:
        # If not, break out of the loop
        break
    
    image = evaluateimg(frame)
    cv2.imshow("Camera Output", image)

    # Check if the 'q' key is pressed to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam (release camera resources)
cap.release()

# Close all OpenCV windows

cv2.destroyAllWindows()

print("classes we found in image", class_list)

'''


'\n\nsetting()\n# Open the default camera (camera index 0)\ncap = cv2.VideoCapture(0)\n# Continuously capture frames from the webcam\nwhile True:\n    # Read a frame from the camera\n    ret, frame = cap.read()\n\n    # Check if the frame was successfully captured\n    if not ret:\n        # If not, break out of the loop\n        break\n    \n    image = evaluateimg(frame)\n    cv2.imshow("Camera Output", image)\n\n    # Check if the \'q\' key is pressed to exit the loop\n    if cv2.waitKey(1) & 0xFF == ord(\'q\'):\n        break\n\n# Release the webcam (release camera resources)\ncap.release()\n\n# Close all OpenCV windows\n\ncv2.destroyAllWindows()\n\nprint("classes we found in image", class_list)\n\n'

In [180]:
def update_camera_feed():
    global class_list
    ret, frame = cap.read()  # Capture a frame from the webcam
    
    if ret:
        class_list.clear()
        # Convert the OpenCV frame to a format suitable for Tkinter
        image = evaluateimg(frame)

        #cv2.imshow('output detection', image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img = PIL.Image.fromarray(img)
        w, h = img.size
        img = img.resize((int(w*img_scale), int(h*img_scale)))
        img = PIL.ImageTk.PhotoImage(image=img)

        # Update the label with the new frame
        label.config(image=img)
        label.image = img
        

    # Schedule the next update
    root.after(10, update_camera_feed)


In [181]:
def toggle_cuda():
    global cuda_enabled
    cuda_enabled = cuda_var.get()
    if cuda_enabled:
        display_masks_checkbox.config(state="normal")  # Enable the checkbox
    else:
        display_masks_checkbox.config(state="disabled")  # Disable the checkbox
    setting()

def update_sc(value):
    global sc
    sc = float(value)
    
def update_img_scale(value):
    global img_scale
    img_scale = float(value)

def update_top_k(value):
    global top_k
    top_k = int(value)

def toggle_display_masks():
    global display_masks_enabled
    display_masks_enabled = display_masks_var.get()

def toggle_display_text_bbx():
    global display_text_bbx_enabled
    display_text_bbx_enabled = display_text_bbx_var.get()
    setting()
    
def update_set_label():
    current_contents = '\n'.join(class_list)  
    set_label.config(text=current_contents)
    root.after(1000, update_set_label)  


In [182]:
setting()
class_list=set()
root = tk.Tk()
root.title("Camera Feed")

cap = cv2.VideoCapture(0)

label = ttk.Label(root)
label.pack(side="right")

#variables
cuda_var = BooleanVar()
display_masks_var = BooleanVar()
display_text_bbx_var = BooleanVar()

cuda_var.set(cuda_enabled)
display_masks_var.set(display_masks_enabled)
display_text_bbx_var.set(display_text_bbx_enabled)

# Checkboxes

cuda_checkbox = ttk.Checkbutton(root, text="CUDA Enable", variable=cuda_var, command=toggle_cuda)
cuda_checkbox.pack()

display_masks_checkbox = ttk.Checkbutton(root, text="Display Masks", variable=display_masks_var, command=toggle_display_masks)
display_masks_checkbox.pack()

display_text_bbx_checkbox = ttk.Checkbutton(root, text="Display Text/BBX", variable=display_text_bbx_var, command=toggle_display_text_bbx)
display_text_bbx_checkbox.pack()

# Sliders
sc_label = ttk.Label(root, text="Select Criteria")
sc_label.pack()
sc_scale = Scale(root, from_=0, to=1, resolution=0.01, orient="horizontal", command=update_sc)
sc_scale.set(sc)
sc_scale.pack()

img_scale_label = ttk.Label(root, text="Image Scale")
img_scale_label.pack()
img_scale_label = Scale(root, from_=0.5, to=2, resolution=0.01, orient="horizontal", command=update_img_scale)
img_scale_label.set(img_scale)
img_scale_label.pack()

top_k_label = ttk.Label(root, text="Top K Boxes")
top_k_label.pack()
top_k_scale = Scale(root, from_=1, to=100, orient="horizontal", command=update_top_k)
top_k_scale.set(top_k)
top_k_scale.pack()

# Create a label for displaying dynamic strings
set_label = ttk.Label(root, text="")
set_label.pack()

update_camera_feed()
update_set_label()

root.mainloop()
cap.release()
cv2.destroyAllWindows()


In [183]:
#if shows pyimageXX error run this and cut all windows and re run
root.mainloop()

In [184]:
class_list

{'person'}